## *nosZ*の機能に必要な遺伝子の同定

In [1]:
from Bio import Phylo
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

In [2]:
# データ前処理
def preprocess(tree, df):
  ## 親ノードの辞書(child node -> parent node)
  parent_dict = {}
  for clade in tree.find_clades(order='level'): # BFS
    if len(parent_dict) == 0: # root
      parent_dict[clade.name] = None

    for child in clade: # 2 elements
      if child.name in parent_dict:
        raise ValueError("Duplicate key: %s" % child.name)
      parent_dict[child.name] = clade.name

  print("# of keys in parent_dict is {}.".format(len(parent_dict))) # archaea: 1801, bacteria: 51753

  ## Possesionの辞書(OG, Node -> Possesion)
  possesion_dict = {}
  for row in df.itertuples():
    possesion_dict[row.OG, row.Node] = row.Possesion

  print("# of keys in possesion_dict is {}.".format(len(possesion_dict))) # this should be equal to # of row in df

  return parent_dict, possesion_dict

In [4]:
# nosZの獲得回数
def count_nosZ_gain(node_list, parent_dict, possesion_dict, threshold=0.5):
  nosZ_gain_count = 0
  for node in node_list:
    src = possesion_dict.get(("K00376", parent_dict[node]), 0.0)
    dst = possesion_dict.get(("K00376", node), 0.0)
    if src < threshold and dst > threshold:
      print("parent_node: {}, child_node: {}".format(parent_dict[node], node))
      print("{}->{}".format(src, dst))
      nosZ_gain_count += 1

  print("nosZ is gained {} times.".format(nosZ_gain_count))

In [6]:
# 分割表を作成
def generate_table(target_OG, candidate_OG, parent_dict, possesion_dict, threshold=0.5):
  # print("\n<OG: {}>".format(candidate_OG))  
  table = np.zeros((2,2))
  
  ## 0のノードはpossesion_dictに登録されていないのでget()で0として取得
  for node in node_list: # 1801 elements
    parent_has_target = possesion_dict.get((target_OG, parent_dict[node]), 0.0)
    if parent_has_target < threshold:  # nosZについて, parentが未獲得
      has_target = possesion_dict.get((target_OG, node), 0.0)
      has_candidate = possesion_dict.get((candidate_OG, parent_dict[node]), 0.0)

      if has_target > threshold and has_candidate > threshold:
        table[0,0] += 1
      elif has_target > threshold and has_candidate < threshold:
        table[0,1] += 1
      elif has_target < threshold and has_candidate > threshold:
        table[1,0] += 1
      elif has_target < threshold and has_candidate < threshold:
        table[1,1] += 1

  # print("# of Non-excluded nodes: {}".format(np.sum(table)))
  return table

In [30]:
### Archaea ###

# データ読み込み
## 系統樹
tree = Phylo.read("../../data/raw/ar122_r202.selected.internal_renamed.tree", "newick")
node_list = [node.name for node in tree.find_clades()]
print(len(node_list)) ## 1801 nodes, which includes 901 terminals.

## 祖先状態推定データ
## Possesion=0の行は削除されている
df = pd.read_table("../../data/raw/ko_gn_possession.archaea.txt", names=["OG", "Node", "Possesion"])
OG_list = df["OG"].unique().tolist()
print(len(OG_list)) ## 4656 OGs

threshold = 0.5
parent_dict, possesion_dict = preprocess(tree, df)
del df, tree
count_nosZ_gain(node_list, parent_dict, possesion_dict, threshold)
## 0→1: 4 times, 0→(0.5 or 1): 7 times, (0 or 0.5)→1: 9 times

1801
4656
# of keys in parent_dict is 1801.
# of keys in possesion_dict is 1690463.
parent_node: node24_, child_node: RS_GCF_000025505.1
0.0->1.0
parent_node: node281_, child_node: RS_GCF_005239435.1
0.0->1.0
parent_node: node813_, child_node: GB_GCA_011373645.1
0.0->1.0
parent_node: node816_, child_node: GB_GCA_002898395.1
0.0->1.0
nosZ is gained 4 times.


In [ ]:
### Bacteria ###
 
# データ読み込み
## 系統樹
# tree = Phylo.read("../../data/raw/bac120_r202.selected.internal_renamed.tree", "newick") ## 51753 nodes
tree = Phylo.read("../../data/itol/bacteria_tree_random.txt", "newick") ## 5999 nodes
node_list = [node.name for node in tree.find_clades()]
print(len(node_list))

## 祖先状態推定データ
## Possesion=0の行は削除されている
df = pd.read_table("../../data/raw/ko_gn_possession.bacteria.txt", names=["OG", "Node", "Possesion"])
OG_list = df["OG"].unique().tolist()
print(len(OG_list))

parent_dict, possesion_dict = preprocess(tree, df)
del df, tree
count_nosZ_obtain(node_list, parent_dict, possesion_dict) ## 4 times

In [31]:
# 検定: nosZを獲得 vs 遺伝子Xを親ノードが保持
target_OG = "K00376" # nosZ

df_result = pd.DataFrame(index=[], columns=["oddsratio", "p-value"])
for candidate_OG in OG_list:
  if candidate_OG != target_OG:
    contingency_table = generate_table(target_OG, candidate_OG, parent_dict, possesion_dict, threshold)
    # print(contingency_table)
    df_result.loc[candidate_OG] = fisher_exact(contingency_table, alternative='two-sided')

print(df_result) 

        oddsratio   p-value
K00003        inf  1.000000
K00004   0.000000  1.000000
K00005   0.000000  1.000000
K00008   6.434783  0.185066
K00010  23.315789  0.057934
...           ...       ...
K24217        NaN  1.000000
K24258        NaN  1.000000
K24393   1.845912  0.486303
K24409   0.000000  0.152038
K24410   0.000000  0.303626

[4655 rows x 2 columns]


In [41]:
# Multiple test correction
alpha = 0.05
p_value_list = df_result["p-value"].to_numpy(copy=True)
df_result["rejected"] = (p_value_list < alpha) # no correction
Bonferroni_rejected, Bf_p_list, _, _ = multipletests(p_value_list, alpha=alpha, method='bonferroni')
BH_rejected, BH_p_list, _, _         = multipletests(p_value_list, alpha=alpha, method='fdr_bh')
BY_rejected, BY_p_list, _, _         = multipletests(p_value_list, alpha=alpha, method='fdr_by')
df_result["Bonferroni adjusted p-value"] = Bf_p_list
df_result["Bonferroni rejected"] = Bonferroni_rejected
df_result["BH adjusted p-value"] = BH_p_list
df_result["BH rejected"] = BH_rejected
df_result["BY adjusted p-value"] = BY_p_list
df_result["BY rejected"] = BY_rejected

print(sum(df_result["rejected"]), sum(df_result["Bonferroni rejected"]), sum(df_result["BH rejected"]), sum(df_result["BY rejected"]))
### alpha=0.05 -> 117 0 0 0


# Merge with ko table
df_result["ko"] = ["ko:" + index for index in df_result.index]
ko_table = pd.read_table("../../data/raw/ko.txt", names=["ko", "description"])
df_merged = pd.merge(ko_table, df_result, on="ko").sort_values("p-value")
del ko_table #, df_result
# df_merged.to_csv("../../data/result/archaea_gain_candidates_sorted.txt", columns=["p-value", "ko", "description"], index=False, sep='\t', float_format="%.4f")
df_merged.head(10)

117 0 0 0


,ko,description,oddsratio,p-value,rejected,Bonferroni adjusted p-value,Bonferroni rejected,BH adjusted p-value,BH rejected,BY adjusted p-value,BY rejected
3835,ko:K16862,PLD6; mitochondrial cardiolipin hydrolase [EC:...,99.285714,0.000717,True,1.0,False,1.0,False,1.0,False
3020,ko:K09796,pccA; periplasmic copper chaperone A,77.111111,0.001128,True,1.0,False,1.0,False,1.0,False
4319,ko:K21147,"moeZR, moeBR; sulfur-carrier protein adenylylt...",68.900000,0.001385,True,1.0,False,1.0,False,1.0,False
331,ko:K00682,GGCT; gamma-glutamylcyclotransferase [EC:4.3.2.9],57.666667,0.001913,True,1.0,False,1.0,False,1.0,False
4387,ko:K21834,dgcB; dimethylglycine catabolism B,inf,0.003044,True,1.0,False,1.0,False,1.0,False
3759,ko:K16016,"rifK, asm24, asm43; 3-amino-5-hydroxybenzoate ...",42.937500,0.003287,True,1.0,False,1.0,False,1.0,False
3574,ko:K14170,pheA; chorismate mutase / prephenate dehydrata...,28.443609,0.003351,True,1.0,False,1.0,False,1.0,False
745,ko:K01638,"aceB, glcB; malate synthase [EC:2.3.3.9]",39.057143,0.003912,True,1.0,False,1.0,False,1.0,False
795,ko:K01715,crt; enoyl-CoA hydratase [EC:4.2.1.17],inf,0.004321,True,1.0,False,1.0,False,1.0,False
1461,ko:K03110,ftsY; fused signal recognition particle receptor,0.028613,0.004799,True,1.0,False,1.0,False,1.0,False
